In [1]:
from scapy.all import rdpcap, UDP #pip install scapy
import numpy as np
import matplotlib.pyplot as plt
import struct

# UDP packet parameters

In [2]:
NCHANS = 256 # Channels per packet
NPOL = 2 # Polarizations
NSPEC_PER_PKT = 16 # Time specs (t0..tn) per packet.
center_channel = 2048  # From the SNAP controllers, channel # 2048 represents sky frequency


# User-Configurable Parameters

In [3]:
sky_freq = 1690e6
NCHAN_START = 1792
number_of_channels=1024
SNAP_sample_rate = 900e6 # For 1.8 GHz this will be 900 MHz.


# Calc Params Based on User Config

In [4]:
channel_width = SNAP_sample_rate/4096
NCHAN_END = NCHAN_START + number_of_channels - 1
NTOTAL = number_of_channels // 256 #total number of channel blocks 
start_freq = sky_freq - (center_channel-NCHAN_START)*channel_width

# Unpack and plot some

In [5]:
fname = './goes17_volt_synced.pcap'
npackets = 5000 # packets to read from pcap file

packets = rdpcap(fname, count=npackets)

In [ ]:
start_capture = 0
volt_x = []
volt_y = []

volt_x_tmp = np.zeros((NSPEC_PER_PKT, NTOTAL*NCHANS), dtype=np.complex64)
volt_y_tmp = np.zeros((NSPEC_PER_PKT, NTOTAL*NCHANS), dtype=np.complex64)

expected_chan_n = NCHAN_START

for i in range(npackets):   
    pkt = bytes(packets[i][UDP].payload)
    pkt_hdr = pkt[:8]
    pkt_data = pkt[8:]
    
    # unpack header
    t = struct.unpack(">Q", pkt_hdr)[0] # ">Q" => big-endian uint64
    version_n = (t >> 56) & 0xFF
    sample_n = (t >> 18) & 0x3fffffffff
    chan_n = (t >> 6) & 0x0FFF
    ant_n = t & 0x3F
    
    # The very first packet to ingest, make sure it's the 
    # first channel number
    if not start_capture: 
        if chan_n != NCHAN_START:
            continue
        else:
            start_capture = 1
            
    
    # unpack 4bit data and reshape array
    data = np.frombuffer(pkt_data, dtype=np.uint8)
    data_real = (data >> 4).astype(np.int8)
    data_imag = 1j*((data << 4) >> 4).astype(np.int8)

        # Deal with signed two's complement values
    for i in range(0,len(data_real)):
        if data_real[i] > 7:
            data_real[i] -= 16
        if data_imag[i] > 7:
            data_imag[i] -= 16j
            
    data = data_real + data_imag
    data = data.reshape(-1, NCHANS, NPOL).astype(np.complex64)    
    
    # This is not clever, as it doesn't check for dropped packets
    # but should do for now
    if chan_n == NCHAN_START:
        volt_x.append(volt_x_tmp.copy())
        volt_y.append(volt_y_tmp.copy())
        
        volt_x_tmp[:] = 0.
        volt_y_tmp[:] = 0.
    
    iband = (chan_n % NCHAN_START)//NCHANS # // enforces integer division in python3
    volt_x_tmp[:,iband*NCHANS:(iband+1)*NCHANS] = data[:,:,0]
    volt_y_tmp[:,iband*NCHANS:(iband+1)*NCHANS] = data[:,:,1]

volt_x = np.concatenate(volt_x)
volt_y = np.concatenate(volt_y)



In [ ]:
plt.figure()
plt.imshow(np.abs(volt_x), interpolation='nearest', aspect='auto')
plt.figure()
plt.imshow(np.abs(volt_y), interpolation='nearest', aspect='auto')

In [ ]:
r_indices=range(NCHAN_START,(NCHAN_START+1024))
indices=list(r_indices)

freqs = []
for i in range(0,1024):
    cur_freq = start_freq + channel_width*i
    freqs.append(cur_freq)

plt.figure()
plt.plot(freqs,np.abs(volt_x).sum(axis=0))
plt.title("Xpol spectrum")

plt.figure()
plt.plot(freqs,np.abs(volt_y).sum(axis=0))
plt.title("Ypol spectrum")